In [1]:
import numpy as np
import keras
import pretty_midi
import librosa

import os
import sys
import mir_eval
import glob

import numpy as np

sys.path.append("..")
sys.path.append("../src")

%load_ext autoreload
%autoreload 
from src import preprocess
from src import constants
from src import datagen
from src.infer import infer_from_processed
from src.metrics import pm_to_pitches_intervals, get_f1_score_notes
from src import helpers


Using TensorFlow backend.


In [2]:
dirname = '../example_data/test_2017/'
preprocess.preprocess(dirname, False)

../example_data/test_2017/2017/MIDI-Unprocessed_053_PIANO053_MID--AUDIO-split_07-06-17_Piano-e_3-04_wav--1.midi
x file is: ../example_data/test_2017/2017/MIDI-Unprocessed_053_PIANO053_MID--AUDIO-split_07-06-17_Piano-e_3-04_wav--1.wav
Audio shape is (3386913,)
after CQT shape is (264, 6616) 
printing first three:
[[1.32677603e-04 1.53895861e-04 1.58763927e-04 ... 1.35217536e-03
  1.30267209e-03 1.32234442e-03]
 [1.70786609e-03 1.70721538e-03 1.73105978e-03 ... 3.04864585e-04
  1.19001187e-04 7.93559087e-05]
 [1.75829896e-03 1.77866492e-03 1.82756736e-03 ... 2.23465326e-03
  2.27342654e-03 2.27657055e-03]]
transposing (6616, 264)
mean is (264,)
std is (264,)
noamrlized is (6616, 264)
outputting x shape (6616, 264)
retrieving times with shape (6616,)
y file is: ../example_data/test_2017/2017/MIDI-Unprocessed_053_PIANO053_MID--AUDIO-split_07-06-17_Piano-e_3-04_wav--1.midi
got piano roll with shape(6616, 88)
leffover padding is 84
(84, 264)
finally getting X  with shape (6700, 264)
finally 

In [2]:
model = '../bin/models/final_model.h5'
test_generator = datagen.DataGenerator(constants.TEST_PROCESSED_DIR, constants.SEQUENCE_SIZE, constants.BATCH_SIZE, constants.CONTEXT_WINDOW_SIZE, 'x_[0-9]+.npy', 'y_[0-9]+.npy')

note_f1s = []
note_precisions = []
note_recalls = []

for sample in glob.glob(os.path.join(constants.TEST_PROCESSED_DIR, 'x_*')):
    isolated_filename = sample.split('/')[-1]
    
    gt_file= os.path.join(constants.TEST_PROCESSED_DIR,test_generator.corresponding_y(isolated_filename))
    referances = np.load(gt_file, mmap_mode='r')
    predictions = infer_from_processed(model, sample)
    
    print(referances.shape)
    print(predictions.shape)
    padded =  np.zeros((predictions.shape[0], 128))
    padded[:, 21:109] = predictions
    padded =  padded.T
    sequence_est = helpers.piano_roll_to_pretty_midi(padded, fs=32, program=0).instruments[0]
    padded =  np.zeros((referances.shape[0], 128))
    padded[:, 21:109] = referances
    padded =  padded.T
    sequence_ref = helpers.piano_roll_to_pretty_midi(padded, fs=32, program=0).instruments[0]
    
    est_intervals,est_pitches = pm_to_pitches_intervals(sequence_est)
    ref_intervals,ref_pitches = pm_to_pitches_intervals(sequence_ref)
    
    note_precision, note_recall, note_f1 = get_f1_score_notes(ref_intervals, ref_pitches, est_intervals, est_pitches)
    note_f1s.append(note_f1)
    note_precisions.append(note_precision)
    note_recalls.append(note_recall)

avg_note_f1 = np.mean(note_f1s)
avg_note_precision = np.mean(note_precisions)
avg_note_recall = np.mean(note_recalls)
print(avg_note_f1)
print(avg_note_precision)
print(avg_note_recall)

x_1.npy
y_1.npy
y_2.npy
x_0.npy
y_0.npy
x_2.npy
data_size is 30500
now listing inputs
['x_1.npy', 'x_0.npy', 'x_2.npy']
now listing outputs
['y_1.npy', 'y_2.npy', 'y_0.npy']
y_1.npy
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
infering from model "../bin/models/final_model.h5"
one up
postprocessing
(14300, 88)
(14300, 88)
y_0.npy
infering from model "../bin/models/final_model.h5"
one up
postprocessing
(6700, 88)
(6700, 88)
y_2.npy
infering from model "../bin/models/final_model.h5"
one up
postprocessing
(9500, 88)
(9500, 88)
0.5378419330206468
0.5955196890819522
0.4930227274937542


In [3]:
referances = np.load('/home/nini/Desktop/Final/project/the-sound-of-music/test_data_proc/y_1.npy', mmap_mode='r')
padded =  np.zeros((referances.shape[0], 128))
padded[:, 21:109] = referances
padded =  padded.T
sequence_ref = helpers.piano_roll_to_pretty_midi(padded, fs=32, program=0)
sequence_ref.write('lookatest.mid')